# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from Config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
#CSV
chart_data = pd.read_csv('../output_data/weather_city.csv')
vac_df = pd.DataFrame(chart_data)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [11]:
# Use the Lat and Lng as locations 
locations = vac_df[["lat", "lon"]]

# Humidity as the weight
humidity = vac_df["humidity"].astype(float)

In [12]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#locate vacation spots in the Philippians
desired_df = vac_df.loc[(vac_df["max temp"] < 80) & (vac_df["max temp"] > 70) & \
                       (vac_df["wind_speed"] < 10) & (vac_df["cloudiness"] == 0)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# set up additional columns to hold information
desired_df["hotel name"] = ""
desired_df.head(20)

c:\users\simo8\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,lat,lon,max temp,humidity,cloudiness,wind_speed,country,date,hotel name
27,vigrestad,-23.12,-134.97,73.80,78,0,5.19,PF,1596334142,
70,port blair,30.76,20.23,76.35,91,0,8.75,LY,1596333857,
90,albany,-8.93,125.41,70.93,38,0,4.29,TL,1596334158,
101,chuy,36.41,64.91,75.00,28,0,1.34,AF,1596334161,
131,east london,14.90,-24.50,77.45,84,0,3.24,CV,1596334343,
147,dingle,36.77,30.72,78.80,83,0,4.70,TR,1596334049,
170,laguna,62.14,117.63,75.11,40,0,5.06,RU,1596334353,
181,kamenka,31.61,34.76,75.00,73,0,5.75,IL,1596334356,
196,jujuy,-20.74,116.85,79.36,20,0,3.40,AU,1596334360,
209,west wendover,-19.01,-57.65,74.62,38,0,2.06,BR,1596334364,


In [30]:
# Store 'Lat' and 'Lng' into  locations 
locations = desired_df[["lat", "lon"]].astype(float)

desired_df_complete = desired_df.dropna()

In [34]:
#Test Dataframe of two lat lng coordinates
lat_lng_test_df = pd.DataFrame({"lat": [49.14, 36.56], "lon": [56.49, 104.21]})

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": test_lat_lng_df, 
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
    }

# use iterrows to iterate through pandas dataframe
for index, row in lat_lng_test_df.iterrows():

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent = 3))
    
    # extract results
    results = response['results']
    
    try:    
        desired_df.loc[index, 'name'] = results[0]['name']
        desired_df.loc[index, 'address'] = results[0]['vicinity']
        desired_df.loc[index, 'price_level'] = results[0]['price_level']
        desired_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
   "html_attributions": [],
   "results": [],
   "status": "INVALID_REQUEST"
}
Missing field/result... skipping.
{
   "html_attributions": [],
   "results": [],
   "status": "INVALID_REQUEST"
}
Missing field/result... skipping.


In [41]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": locations, 
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in desired_df.iterrows():

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:    
        desired_df.loc[index, 'name'] = results[0]['name']
        desired_df.loc[index, 'address'] = results[0]['vicinity']
        desired_df.loc[index, 'price_level'] = results[0]['price_level']
        desired_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

NameError: name 'restr_type' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
